In [2]:
import talib
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
import yfinance as yf
import plotly.express as px
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta, date
import backtesting
from backtesting import Backtest, Strategy
import warnings
import itertools

Выбираем 9 финансовых инструментов для анализа: "AMZN", "GOOG", "TSLA", "MBG.DE", "DTG.DE", "VOW.DE", "BTC-USD", "ETH-USD", "SOL-USD" (три американские акции, три немецкие и три криптовалюты).  
Интервал - 1 день, период - с 01 февраля 2022 до вчерашнего дня.  
Скачиваем данные с YAHOO и складываем их в словарь df_dict.  
В дальнейшем можно сделать словарь словарей для хранения технических индикаторов и других данный по инструменту в 
одном словаре.

In [10]:
df_dict = {}
tickers = ["AMZN", "GOOG", "TSLA", "MBG.DE", "DTG.DE", "VOW.DE", "BTC-USD", "ETH-USD", "SOL-USD"]
yesterday = date.today() - timedelta(days = 1)

for ticker in tickers:

    df_dict[ticker] = yf.download(ticker, start='2022-02-01', end=yesterday, interval='1d').drop(columns=['Adj Close'])
    #df_dict[ticker].index = pd.to_datetime(df_dict[ticker].index)
    df_dict[ticker].columns = [f'{Price}' for Price, Ticker in df_dict[ticker].columns]
    df_dict[ticker] = df_dict[ticker].reset_index()
    df_dict[ticker].columns = df_dict[ticker].columns.str.lower()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [11]:
class TemaMacdStrategy(Strategy):
    
    def init(self):
        self.signal = self.I(lambda: self.data.Signal)
        self.previous_signal = 0
        self.size = 0.1

    def next(self):
        current_signal = self.signal[-1]

        if current_signal != self.previous_signal:
            if current_signal == 1:
                if self.position.is_short:
                    self.position.close()
                    
                if not self.position.is_long:
                    self.buy(size=self.size)
                    
            elif current_signal == -1:
                if self.position.is_long:
                    self.position.close()
                   
                if not self.position.is_short:
                    self.sell(size=self.size)
                    
            elif current_signal == 0:
                if self.position:
                    self.position.close()

        self.previous_signal = current_signal

### Walk Forward Optimization

In [13]:
# напишем функцию стартегии
def apply_strategy(data, params):
    """
    Применение стратегии с оптимизированными параметрами.

    :param df: DataFrame с данными, на которые будут наложены индикаторы.
    :param params: Словарь с оптимизированными параметрами.
    :return: DataFrame с рассчитанными индикаторами и сигналами.
    """
    df = data.copy()
    # Извлекаем параметры из словаря
    rsi_period = params['rsi_period']
    #tema_period = params['tema_period']
    fastMACD_period = params['fastMACD_period']
    slowMACD_period = params['slowMACD_period']
    signalMACD_period = params['signalMACD_period']
    
    # Добавляем индикаторы
    #df['tema'] = talib.TEMA(df['close'], timeperiod=tema_period)
    df['macd'], df['macd_signal'], df['macd_hist'] = talib.MACD(df['close'], fastperiod=fastMACD_period, slowperiod=slowMACD_period, signalperiod=signalMACD_period)
    df['rsi'] = talib.RSI(df['close'], timeperiod=rsi_period)


    # Создаем сигналы для покупки и продажи
    df['signal'] = 0
    df.loc[(df['macd'] > df['macd_signal']) & (50 <= df['rsi']) & (df['rsi'] < 70), 'signal'] = 1  # Сигнал на покупку
    df.loc[(df['macd'] < df['macd_signal']) & (30 < df['rsi']) & (df['rsi'] < 50), 'signal'] = -1  # Сигнал на продажу
    
    return df[["date", "open", "high", "low", "close", "volume","signal"]]

def bactest_strategy(df, strategy_class, params, plot=False):
    """
    Запускает бэктест с переданными параметрами стратегии.

    :param df: DataFrame с данными для бэктеста.
    :param strategy_class: Класс стратегии для бэктеста.
    :param params: Словарь с параметрами стратегии.
    :return: Статистика бэктеста.
    """
    # Применяем стратегию с переданными параметрами
    
    df = apply_strategy(df.copy(), params)
    
    # Подготовка данных для бэктеста
    bt_df = df.copy()
    bt_df.columns = bt_df.columns.str.capitalize()
    bt_df.rename(columns={'Date': 'Datetime'}, inplace=True)
    bt_df["Datetime"] = pd.to_datetime(bt_df["Datetime"])
    bt_df.set_index('Datetime', inplace=True)

    # Создаем объект класса Backtest с текущей стратегией
    bt = Backtest(bt_df, strategy_class, cash=500000, commission=.002, exclusive_orders=True, margin=0.1)

    # Запускаем бэктест
    stats = bt.run()
    if plot:
        bt.plot(
        plot_equity=True,
        plot_drawdown=True,
        relative_equity=False,
        )
    return stats

def get_best_strategy(buffer, strategy_class):


    # Задаем возможные значения для параметров стратегии
    rsi_period_list = [12, 14]
    fastMACD_period_list = [11, 12]
    slowMACD_period_list = [25, 26]
    signalMACD_period_list = [7, 8]

    # Для хранения лучших параметров и лучшего результата
    best_params = None
    best_performance = -float('inf')

    # Проходим по всем комбинациям параметров
    for rsi_period, fastMACD_period, slowMACD_period, signalMACD_period in itertools.product(rsi_period_list, fastMACD_period_list, slowMACD_period_list, signalMACD_period_list):
        
        # Создаем словарь с текущими параметрами
        params = {
            'rsi_period': rsi_period,
            'fastMACD_period': fastMACD_period,
            'slowMACD_period': slowMACD_period,
            'signalMACD_period': signalMACD_period
        }
        

        # Запускаем бэктест с текущими параметрами
        stats = bactest_strategy(buffer.copy(), strategy_class, params)

        # Определяем метрику, по которой будем выбирать лучшую стратегию 
        performance = stats['Sharpe Ratio'] 
        
        # Сравниваем с лучшим результатом и сохраняем лучшие параметры
        if performance > best_performance:
            best_performance = performance
            best_params = params

    # print(f"Best Performance: {best_performance}")
    # print(f"Best Parameters: {best_params}")
    return best_params

    

In [ ]:
btest_res = pd.DataFrame()
for ticker in tickers:

    df = df_dict[ticker].copy()
    train_size = 90  # Размер окна тренировки
    test_size = 30    # Размер тестового окна

    # Инициализация DataFrame для сигналов
    signals_df = pd.DataFrame()

    # Определение количества итераций
    num_iterations = (len(df) - train_size) // test_size

    for i in range(num_iterations + 1):
        # Определение границ обучающего и тестового окон
        start_train = i * test_size
        end_train = start_train + train_size
        start_test = end_train
        end_test = start_test + test_size
        
        # Если конец тестового окна выходит за пределы данных, обрезаем его
        if end_test > len(df):
            end_test = len(df)
        
        # Определяем окна для тренировки и тестирования
        train_data = df.iloc[start_train:end_train].copy()
        test_data = df.iloc[start_test:end_test].copy()
        
        
        # Оптимизация на тренировочном окне
        best_params = get_best_strategy(train_data, TemaMacdStrategy)
        
        # Объединяем данные тренировки и теста
        combined_data = pd.concat([train_data, test_data]).reset_index(drop=True)
        
        # Применяем стратегию на объединенном окне с оптимальными параметрами
        combined_with_signal = apply_strategy(combined_data.copy(), best_params)
        
        # Извлекаем только часть данных, относящуюся к тестовому окну
        test_with_signal = combined_with_signal.iloc[-test_size:].copy()
        
        # Добавляем сигналы из тестового окна в signals_df
        signals_df = pd.concat([signals_df, test_with_signal], ignore_index=True)

    bt_df = signals_df.copy()
    bt_df.columns = bt_df.columns.str.capitalize()
    bt_df.rename(columns={'Date': 'Datetime'}, inplace=True)
    bt_df["Datetime"] = pd.to_datetime(bt_df["Datetime"])
    bt_df.set_index('Datetime', inplace=True)
    bt_df = bt_df.sort_index()

    bt = Backtest(bt_df, TemaMacdStrategy, cash=500000, commission=0.002, exclusive_orders=True, margin=0.1)

        # Запускаем бэктест
    stats = bt.run()
    btest_res[ticker] = stats[:25]
    # print(ticker)
    # print(stats[:25])


In [17]:
btest_res

,AMZN,GOOG,TSLA,MBG.DE,DTG.DE,VOW.DE,BTC-USD,ETH-USD,SOL-USD
Start,2022-06-10 00:00:00+00:00,2022-06-10 00:00:00+00:00,2022-06-10 00:00:00+00:00,2022-06-09 00:00:00+00:00,2022-06-09 00:00:00+00:00,2022-06-09 00:00:00+00:00,2022-05-02 00:00:00+00:00,2022-05-02 00:00:00+00:00,2022-05-02 00:00:00+00:00
End,2024-12-11 00:00:00+00:00,2024-12-11 00:00:00+00:00,2024-12-11 00:00:00+00:00,2024-12-11 00:00:00+00:00,2024-12-11 00:00:00+00:00,2024-12-11 00:00:00+00:00,2024-12-11 00:00:00+00:00,2024-12-11 00:00:00+00:00,2024-12-11 00:00:00+00:00
Duration,915 days 00:00:00,915 days 00:00:00,915 days 00:00:00,916 days 00:00:00,916 days 00:00:00,916 days 00:00:00,954 days 00:00:00,954 days 00:00:00,954 days 00:00:00
Exposure Time [%],73.484848,75.757576,64.848485,66.666667,65.30303,68.787879,60.520833,65.3125,62.8125
Equity Final [$],598150.784203,412935.697486,1323051.600323,523012.7031,599285.07176,268876.143,354285.013158,568490.684179,1661016.479771
Equity Peak [$],716491.44859,500188.500847,1495175.033845,603468.850269,731165.249162,563426.124685,591280.010992,1195052.805789,1711566.460219
Return [%],19.630157,-17.412861,164.61032,4.602541,19.857014,-46.224771,-29.142997,13.698137,232.203296
Buy & Hold Return [%],109.995432,76.536322,82.909183,-16.81852,26.944684,-57.598855,162.587063,34.13621,159.698788
Return (Ann.) [%],7.432645,-7.367162,47.585167,1.773593,7.333089,-21.523592,-12.336975,5.028913,58.226798
Volatility (Ann.) [%],30.077695,24.260253,65.286751,21.252505,22.768383,18.332258,32.139003,52.570681,114.97019


Нарисуем для сравнения все 9 графиков отношения цены закрытия к средней цене закрытия за наш период наблюдений. 
Видим рост волатильности с разной скоростью у разных инструментов с февраля 2024.

In [21]:
for ticker in tickers:

    df_dict[ticker].index = pd.to_datetime(df_dict[ticker].date)

In [22]:
fig = go.Figure()

# Добавляем графики
for ticker in tickers:
    price_mean = df_dict[ticker].close.mean()
    fig.add_trace(go.Scatter(x=df_dict[ticker].index, y=(df_dict[ticker].close)/price_mean,                         
                             mode='lines', name=ticker,
                             line=dict(width=1)))

# Настройка осей и названий
fig.update_layout(title='Interval volatility',
                  xaxis_title='Date',
                  yaxis_title='Close Price/price_mean',
                  template='plotly_dark')

# Отображаем график
fig.show()